In [ ]:
import os
import numpy as np
from numpy import zeros
from numpy import ones
from numpy.random import randint
import tensorflow as tf
from tensorflow.image import psnr
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from matplotlib import pyplot as plt
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img

from tensorflow.keras.layers import Reshape
from tensorflow.keras import layers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def mae_loss(y_true, y_pred):

  mae = tf.reduce_mean(tf.abs(y_true - y_pred))
  return mae

In [ ]:
def psnr_metric(y_true, y_pred):

    y_true = (y_true + 1)/2 * 255
    y_pred = (y_pred + 1)/2 * 255

    return tf.image.psnr(y_true, y_pred, max_val=255)

In [ ]:
def define_discriminator(image_shape):

	# weight initialization
	init = RandomNormal(stddev=0.02)

	# source image input
	in_src_image = Input(shape=image_shape)
	# target image input
	in_target_image = Input(shape=image_shape)

	# concatenate images, channel-wise
	merged = tf.concat([in_src_image, in_target_image], axis = -1)

	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)

	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)

	# patch output
	d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	patch_out = Activation('linear')(d)

	# define model
	model = Model(merged, patch_out)

	return model

In [ ]:
def gradient_penalty(discriminator, real_images, fake_images, in_src_image):
    real_concat = tf.concat([in_src_image, real_images], axis=-1)
    fake_concat = tf.concat([in_src_image, fake_images], axis=-1)

    # Compute interpolations
    alpha = tf.random.uniform(shape=[real_images.shape[0], 1, 1, 1], minval=0., maxval=1.)
    interpolated = alpha * real_concat + (1 - alpha) * fake_concat

    with tf.GradientTape() as tape:
        tape.watch(interpolated)
        pred = discriminator(interpolated, training=True)

    grads = tape.gradient(pred, interpolated)
    norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
    gradient_penalty = tf.reduce_mean((norm - 1.0) ** 2)
    return gradient_penalty

In [ ]:
def define_encoder_block(layer_in, n_filters, batchnorm=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add downsampling layer
	g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# conditionally add batch normalization
	if batchnorm:
		g = BatchNormalization()(g, training=True)
	# leaky relu activation
	g = LeakyReLU(alpha=0.2)(g)

	return g

In [ ]:
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add upsampling layer
	g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# add batch normalization
	g = BatchNormalization()(g, training=True)
	# conditionally add dropout
	if dropout:
		g = Dropout(0.5)(g, training=True)
	# merge with skip connection
	g = Concatenate()([g, skip_in])
	# relu activation
	g = Activation('relu')(g)
	return g

In [ ]:
def define_generator(image_shape=(256,256,3)):

	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# encoder model: C64-C128-C256-C512-C512-C512-C512-C512
	e1 = define_encoder_block(in_image, 64, batchnorm=False)
	e2 = define_encoder_block(e1, 128)
	e3 = define_encoder_block(e2, 256)
	e4 = define_encoder_block(e3, 512)
	e5 = define_encoder_block(e4, 512)
	e6 = define_encoder_block(e5, 512)
	e7 = define_encoder_block(e6, 512)
	# bottleneck, no batch norm and relu
	b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
	b = Activation('relu')(b)
	# decoder model: CD512-CD512-CD512-C512-C256-C128-C64
	d1 = decoder_block(b, e7, 512)
	d2 = decoder_block(d1, e6, 512)
	d3 = decoder_block(d2, e5, 512)
	d4 = decoder_block(d3, e4, 512, dropout=False)
	d5 = decoder_block(d4, e3, 256, dropout=False)
	d6 = decoder_block(d5, e2, 128, dropout=False)
	d7 = decoder_block(d6, e1, 64, dropout=False)
	# output
	g = Conv2DTranspose(image_shape[2], (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
	out_image = Activation('tanh')(g)  #Generates images in the range -1 to 1. So change inputs also to -1 to 1
	# define model
	model = Model(in_image, out_image)
	return model

In [ ]:
def generate_real_samples(dataset, n_samples, patch_shape):
	# unpack dataset
	trainA, trainB = dataset
	# choosing random instances
	ix = randint(0, trainA.shape[0], n_samples)
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels (1)
	#y = ones((n_samples, patch_shape, patch_shape, 1))
	y = tf.random.uniform((n_samples, patch_shape, patch_shape, 1), minval=0.8, maxval=1.0)
	return [X1, X2], y

In [ ]:
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model(samples, training = True)
	# create 'fake' class labels (0)
	#y = zeros((len(X), patch_shape, patch_shape, 1))
	y = tf.random.uniform((len(X), patch_shape, patch_shape, 1), minval=0.0, maxval=0.2)
	return X, y

In [ ]:
def summarize_performance(step, g_model, d_model, dataset, n_samples=3):
  # select a sample of input images
  [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
  # generate a batch of fake samples
  X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
  # scale all pixels from [-1,1] to [0,1]
  X_realA = (X_realA + 1) / 2.0
  X_realB = (X_realB + 1) / 2.0
  X_fakeB = (X_fakeB + 1) / 2.0
  # plot real source images
  for i in range(n_samples):
    plt.subplot(3, n_samples, 1 + i)
    plt.axis('off')
    plt.imshow(X_realA[i])
  # plot generated target image
  for i in range(n_samples):
    plt.subplot(3, n_samples, 1 + n_samples + i)
    plt.axis('off')
    plt.imshow(X_fakeB[i])
  # plot real target image
  for i in range(n_samples):
    plt.subplot(3, n_samples, 1 + n_samples*2 + i)
    plt.axis('off')
    plt.imshow(X_realB[i])
  # save plot to file
  filename1 = '/content/drive/MyDrive/Image_Denoising/plot_GAN_L4_WGP_Transpose%06d.png' % (step+1)
  plt.savefig(filename1)
  plt.close()
  # save the generator model
  filename2 = '/content/drive/MyDrive/Image_Denoising/GAN_model_GITHUB_L4_WGP_Transpose%06d' % (step+1)
  g_model.save(os.path.join(filename2, 'g_model_GITHUB_L4_WGP_Transpose%06d.h5' % (step+1)))
  d_model.save(os.path.join(filename2, 'd_model_GITHUB_L4_WGP_Transpose%06d.h5' % (step+1)))
  print('>Saved: %s and %s' % (filename1, filename2))

In [ ]:
def train(d_model, g_model, dataset, test_dataset, n_epochs=100, n_batch=1, n_critics = 3, val_batch_size = 20, mae_coeff = 100, lambda_gp = 10):
  # determine the output square shape of the discriminator
  n_patch = d_model.output_shape[1]
  # unpack dataset
  trainA, trainB = dataset
  # calculate the number of batches per training epoch
  bat_per_epo = int(len(trainA) / n_batch)
  # calculate the number of training iterations
  n_steps = bat_per_epo * n_epochs
  # creating a file to track loss values
  filename = '/content/drive/MyDrive/Image_Denoising/GAN_Losses_L4_WGP_Transpose.txt'
  filename_after_epoch = '/content/drive/MyDrive/Image_Denoising/GAN_LossesAfterEpoch_L4_WGP_Transpose.txt'

  # select a batch of real val samples
  [X_real_valA, X_real_valB], _ = generate_real_samples(test_dataset, val_batch_size, n_patch)

  psnr_avg = 0

  d_loss_avg = 0
  g_loss_avg = 0

  # lr reduce after 6 epochs
  lr_schedule_generator = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1.2e-4,
    decay_steps=bat_per_epo*6,
    decay_rate=0.7,
    staircase=True
  )
  # lr reduce after 6 epochs
  lr_schedule_discriminator = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=bat_per_epo*6,
    decay_rate=0.7,
    staircase=True
  )

  # Optimizers
  discriminator_optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule_discriminator)
  generator_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule_generator, beta_1=0.5)


  # manually enumerate epochs
  for i in range(n_steps):
    step_for_avg = (i % (bat_per_epo))
    for _ in range(n_critics):
      with tf.GradientTape() as tape:
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        # generate a batch of fake samples
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
        X_realA = tf.convert_to_tensor(X_realA)
        X_realB = tf.convert_to_tensor(X_realB)
        X_fakeB = tf.convert_to_tensor(X_fakeB)
        y_real = tf.convert_to_tensor(y_real)
        y_fake = tf.convert_to_tensor(y_fake)
        X_in_real = tf.concat([X_realA, X_realB], axis = -1)
        X_in_fake = tf.concat([X_realA, X_fakeB], axis = -1)

        real_output = d_model(X_in_real, training = True)
        fake_output = d_model(X_in_fake, training = True)

        d_loss = tf.reduce_mean(fake_output) - tf.reduce_mean(real_output)
        gp_loss = gradient_penalty(d_model, X_realB, X_fakeB, X_realA)
        d_loss += lambda_gp*gp_loss

        d_loss_avg = (step_for_avg*(d_loss_avg) + d_loss)/(step_for_avg+1)

      gradients_d = tape.gradient(d_loss, d_model.trainable_variables)

      # Clip gradients
      gradients_d = [tf.clip_by_norm(g, 1.0) for g in gradients_d]

      discriminator_optimizer.apply_gradients(zip(gradients_d, d_model.trainable_variables))


    # Update generator
    # select a batch of real samples
    [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
    # generate a batch of fake samples
    #X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
    X_realA = tf.convert_to_tensor(X_realA)
    X_realB = tf.convert_to_tensor(X_realB)
    y_real = tf.convert_to_tensor(y_real)
    with tf.GradientTape() as tape:
      tape.watch(X_realA)
      X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
      X_fakeB = tf.convert_to_tensor(X_fakeB)
      y_fake = tf.convert_to_tensor(y_fake)
      X_in_fake = tf.concat([X_realA, X_fakeB], axis = -1)
      fake_output = d_model(X_in_fake, training=False)
      g_loss = -tf.reduce_mean(fake_output)
      mae_loss_value = mae_loss(X_realB, X_fakeB)
      g_loss += mae_coeff * mae_loss_value

      g_loss_avg = (step_for_avg*(g_loss_avg) + g_loss)/(step_for_avg+1)

    generator_gradients = tape.gradient(g_loss, g_model.trainable_variables)

    generator_gradients = [tf.clip_by_norm(g, 1.0) for g in generator_gradients]

    generator_optimizer.apply_gradients(zip(generator_gradients, g_model.trainable_variables))

    psnr = psnr_metric(X_realB, X_fakeB)
    psnr = tf.math.reduce_mean(psnr)
    psnr_avg = (step_for_avg*(psnr_avg) + psnr)/(step_for_avg+1)

    # summarize performance
    print(f'>%d, d[%.3f] g[%.3f] d_loss_avg {d_loss_avg} g_loss_avg {g_loss_avg} psnr {psnr} psnr_avg {psnr_avg}' % (i+1, d_loss, g_loss))

    with open(filename, 'a') as file:
      file.write(f'At step {i+1} d_loss {d_loss} g_loss {g_loss} d_loss_avg {d_loss_avg} g_loss_avg {g_loss_avg} psnr {psnr} psnr_avg {psnr_avg}\t')

    # summarize model performance
    if (i+1) % (bat_per_epo) == 0:
      summarize_performance(i, g_model, d_model, dataset)

      psnr_val = 0

      # generate a batch of fake val samples
      X_fake_valB, _ = generate_fake_samples(g_model, X_real_valA, n_patch)

      X_in_val_real = tf.concat([X_real_valA, X_real_valB], axis = -1)
      X_in_val_fake = tf.concat([X_real_valA, X_fake_valB], axis = -1)
      real_output_val = d_model(X_in_val_real)
      fake_output_val = d_model(X_in_val_fake)

      d_loss_val = tf.reduce_mean(fake_output_val) - tf.reduce_mean(real_output_val)
      gp_loss_val = gradient_penalty(d_model, X_real_valB, X_fake_valB, X_real_valA)
      d_loss_val += lambda_gp*gp_loss_val


      g_loss_val = -tf.reduce_mean(fake_output_val)
      mae_loss_val = mae_loss(X_real_valB, X_fake_valB)
      g_loss_val += mae_coeff * mae_loss_val

      for k in range(val_batch_size):
        psnr_val += psnr_metric(X_real_valB[k], X_fake_valB[k])
      psnr_val = (psnr_val)/(val_batch_size)

      print(f'After {i+1}th step psnr_avg {psnr_avg} and psnr_val {psnr_val}')

      with open(filename_after_epoch, 'a') as file:
        file.write(f'At step {i+1} d_loss {d_loss} g_loss {g_loss} d_loss_avg {d_loss_avg} g_loss_avg {g_loss_avg} d_loss_val {d_loss_val} g_loss_val {g_loss_val} psnr {psnr} psnr_avg {psnr_avg} psnr_val {psnr_val}\n')

      psnr_avg = 0
      d_loss_avg = 0
      g_loss_avg = 0

    if (i+1) % (bat_per_epo/4) == 0:
      # generate a batch of fake val samples
      X_fake_valB, _ = generate_fake_samples(g_model, X_real_valA, n_patch)

      X_in_val_real = tf.concat([X_real_valA, X_real_valB], axis = -1)
      X_in_val_fake = tf.concat([X_real_valA, X_fake_valB], axis = -1)
      real_output_val = d_model(X_in_val_real)
      fake_output_val = d_model(X_in_val_fake)

      d_loss_val = tf.reduce_mean(fake_output_val) - tf.reduce_mean(real_output_val)
      gp_loss_val = gradient_penalty(d_model, X_real_valB, X_fake_valB, X_real_valA)
      d_loss_val += lambda_gp*gp_loss_val

      g_loss_val = -tf.reduce_mean(fake_output_val)
      mae_loss_val = mae_loss(X_real_valB, X_fake_valB)
      g_loss_val += mae_coeff * mae_loss_val
      psnr_val = 0
      for k in range(val_batch_size):
        psnr_val += psnr_metric(X_real_valB[k], X_fake_valB[k])
      psnr_val = (psnr_val)/(val_batch_size)

      print(f'D_LOSS_VAL {d_loss_val} G_LOSS_VAL {g_loss_val} PSNR_VALIDATION is: {psnr_val}')


In [ ]:
def load_images(path):
  images = list()
  files = os.listdir(path)
  files.sort()

  for filename in files:
    pixels = load_img(os.path.join(path, filename), target_size = (256, 256))
    pixels = img_to_array(pixels)
    images.append(pixels)

  return np.asarray(images)

In [ ]:
low_train_images = load_images('/content/drive/MyDrive/Image_Denoising/New_train/train/low')
low_test_images = load_images('/content/drive/MyDrive/Image_Denoising/New_train/test/low')
high_train_images = load_images('/content/drive/MyDrive/Image_Denoising/New_train/train/high')
high_test_images = load_images('/content/drive/MyDrive/Image_Denoising/New_train/test/high')

In [ ]:
low_train_images.shape

(1552, 256, 256, 3)

In [ ]:
low_test_images.shape

(388, 256, 256, 3)

In [ ]:
high_train_images.shape

(1552, 256, 256, 3)

In [ ]:
high_test_images.shape

(388, 256, 256, 3)

In [ ]:
def pre_process(data):
  X1, X2 = data[0], data[1]

  X1 = (X1 - 127.5) / 127.5
  X2 = (X2 - 127.5) / 127.5

  return [X1, X2]

In [ ]:
data = [low_train_images, high_train_images]

In [ ]:
dataset = pre_process(data)

In [ ]:
test_data = [low_test_images, high_test_images]

In [ ]:
test_dataset = pre_process(test_data)

In [ ]:
image_shape = low_train_images.shape[1:]

d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [ ]:
g_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_6 (Conv2D)           (None, 128, 128, 64)         3136      ['input_4[0][0]']             
                                                                                                  
 leaky_re_lu_5 (LeakyReLU)   (None, 128, 128, 64)         0         ['conv2d_6[0][0]']            
                                                                                                  
 conv2d_7 (Conv2D)           (None, 64, 64, 128)          131200    ['leaky_re_lu_5[0][0]']       
                                                                                            

In [ ]:
d_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 256, 256, 6)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 64)      6208      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 128, 128, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 128)       131200    
                                                                 
 batch_normalization (Batch  (None, 64, 64, 128)       512       
 Normalization)                                                  
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 64, 64, 128)       0         
                                                             

In [ ]:
from datetime import datetime
start1 = datetime.now()

train(d_model, g_model, dataset, test_dataset, n_epochs = 100, n_batch = 4, n_critics= 5, val_batch_size = 20)

end1 = datetime.now()

>1, d[29526.371] g[39.298] d_loss_avg 29526.37109375 g_loss_avg 39.297691345214844 psnr 12.51797866821289 psnr_avg 12.51797866821289
>2, d[71527.719] g[33.365] d_loss_avg 67551.1640625 g_loss_avg 36.33148956298828 psnr 13.785327911376953 psnr_avg 13.151653289794922
>3, d[35101.250] g[30.284] d_loss_avg 50426.34375 g_loss_avg 34.31560516357422 psnr 14.669147491455078 psnr_avg 13.657485008239746
>4, d[92778.414] g[31.711] d_loss_avg 59774.2265625 g_loss_avg 33.66448211669922 psnr 14.352278709411621 psnr_avg 13.831183433532715
>5, d[64059.148] g[39.862] d_loss_avg 59300.79296875 g_loss_avg 34.90398406982422 psnr 12.686873435974121 psnr_avg 13.602320671081543
>6, d[35218.441] g[35.578] d_loss_avg 50461.48828125 g_loss_avg 35.01633834838867 psnr 13.653539657592773 psnr_avg 13.610857963562012
>7, d[46219.102] g[41.164] d_loss_avg 55368.8359375 g_loss_avg 35.894622802734375 psnr 12.494061470031738 psnr_avg 13.451315879821777
>8, d[30066.475] g[38.179] d_loss_avg 45666.2265625 g_loss_avg 36.18

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


>Saved: /content/drive/MyDrive/Image_Denoising/plot_GAN_L4_WGP_Transpose000388.png and /content/drive/MyDrive/Image_Denoising/GAN_model_GITHUB_L4_WGP_Transpose000388
After 388th step psnr_avg 16.147558212280273 and psnr_val 17.613754272460938
D_LOSS_VAL 419.54217529296875 G_LOSS_VAL 24.159542083740234 PSNR_VALIDATION is: 17.607891082763672
>389, d[575.184] g[17.340] d_loss_avg 575.1842041015625 g_loss_avg 17.33969497680664 psnr 19.756685256958008 psnr_avg 19.756685256958008
>390, d[379.689] g[16.941] d_loss_avg 407.91900634765625 g_loss_avg 17.140201568603516 psnr 20.365325927734375 psnr_avg 20.061004638671875
>391, d[442.199] g[14.341] d_loss_avg 446.2839660644531 g_loss_avg 16.20725440979004 psnr 21.09859848022461 psnr_avg 20.406869888305664
>392, d[315.974] g[21.915] d_loss_avg 390.78509521484375 g_loss_avg 17.63420295715332 psnr 17.626953125 psnr_avg 19.711891174316406
>393, d[431.083] g[23.884] d_loss_avg 459.15570068359375 g_loss_avg 18.884157180786133 psnr 18.07278060913086 psnr

>Saved: /content/drive/MyDrive/Image_Denoising/plot_GAN_L4_WGP_Transpose000776.png and /content/drive/MyDrive/Image_Denoising/GAN_model_GITHUB_L4_WGP_Transpose000776
After 776th step psnr_avg 18.57069206237793 and psnr_val 18.178010940551758
D_LOSS_VAL 4.7679123878479 G_LOSS_VAL 22.649824142456055 PSNR_VALIDATION is: 18.1551456451416
>777, d[5.386] g[15.407] d_loss_avg 5.385721206665039 g_loss_avg 15.407098770141602 psnr 20.886398315429688 psnr_avg 20.886398315429688
>778, d[10.069] g[22.291] d_loss_avg 10.809537887573242 g_loss_avg 18.84915542602539 psnr 18.95413589477539 psnr_avg 19.92026710510254
>779, d[4.318] g[22.899] d_loss_avg 6.274633407592773 g_loss_avg 20.19901466369629 psnr 17.64183807373047 psnr_avg 19.160791397094727
>780, d[3.140] g[19.112] d_loss_avg 4.1236090660095215 g_loss_avg 19.927223205566406 psnr 18.994159698486328 psnr_avg 19.11913299560547
>781, d[2.995] g[14.960] d_loss_avg 4.971858501434326 g_loss_avg 18.933828353881836 psnr 21.402658462524414 psnr_avg 19.575

>Saved: /content/drive/MyDrive/Image_Denoising/plot_GAN_L4_WGP_Transpose001164.png and /content/drive/MyDrive/Image_Denoising/GAN_model_GITHUB_L4_WGP_Transpose001164
After 1164th step psnr_avg 18.90369987487793 and psnr_val 18.45008659362793
D_LOSS_VAL 0.014285590499639511 G_LOSS_VAL 22.153921127319336 PSNR_VALIDATION is: 18.40642547607422
>1165, d[0.747] g[20.789] d_loss_avg 0.7472177743911743 g_loss_avg 20.78887367248535 psnr 19.219921112060547 psnr_avg 19.219921112060547
>1166, d[0.338] g[16.613] d_loss_avg 0.31550705432891846 g_loss_avg 18.700769424438477 psnr 19.85262680053711 psnr_avg 19.536273956298828
>1167, d[0.037] g[19.236] d_loss_avg 0.08554825931787491 g_loss_avg 18.87922477722168 psnr 19.529541015625 psnr_avg 19.534029006958008
>1168, d[0.012] g[30.073] d_loss_avg 0.18377603590488434 g_loss_avg 21.67777442932129 psnr 15.990153312683105 psnr_avg 18.648059844970703
>1169, d[0.001] g[19.734] d_loss_avg 0.10550813376903534 g_loss_avg 21.289108276367188 psnr 18.634044647216797

...

>Saved: /content/drive/MyDrive/Image_Denoising/plot_GAN_L4_WGP_Transpose012028.png and /content/drive/MyDrive/Image_Denoising/GAN_model_GITHUB_L4_WGP_Transpose012028
After 12028th step psnr_avg 22.934680938720703 and psnr_val 21.118127822875977
D_LOSS_VAL 0.0394449420273304 G_LOSS_VAL 16.620792388916016 PSNR_VALIDATION is: 21.06818199157715
>12029, d[1.160] g[9.031] d_loss_avg 1.1601243019104004 g_loss_avg 9.030545234680176 psnr 24.97740936279297 psnr_avg 24.97740936279297
>12030, d[0.064] g[8.272] d_loss_avg 0.19007465243339539 g_loss_avg 8.651113510131836 psnr 25.2923641204834 psnr_avg 25.1348876953125
>12031, d[0.060] g[8.692] d_loss_avg 0.08652877062559128 g_loss_avg 8.664731979370117 psnr 24.886432647705078 psnr_avg 25.05206871032715
>12032, d[0.101] g[10.508] d_loss_avg 0.05446036159992218 g_loss_avg 9.125465393066406 psnr 24.088035583496094 psnr_avg 24.811059951782227
>12033, d[0.024] g[7.379] d_loss_avg 0.039121806621551514 g_loss_avg 8.776103019714355 psnr 26.35574722290039 ps

>Saved: /content/drive/MyDrive/Image_Denoising/plot_GAN_L4_WGP_Transpose012416.png and /content/drive/MyDrive/Image_Denoising/GAN_model_GITHUB_L4_WGP_Transpose012416
After 12416th step psnr_avg 23.193994522094727 and psnr_val 20.70767593383789
D_LOSS_VAL 0.07009837031364441 G_LOSS_VAL 17.25436782836914 PSNR_VALIDATION is: 20.771381378173828
>12417, d[0.142] g[13.350] d_loss_avg 0.141626238822937 g_loss_avg 13.3495454788208 psnr 22.03907585144043 psnr_avg 22.03907585144043
>12418, d[0.132] g[16.526] d_loss_avg 0.1450517177581787 g_loss_avg 14.937704086303711 psnr 20.64826202392578 psnr_avg 21.343669891357422
>12419, d[0.058] g[10.950] d_loss_avg 0.09397705644369125 g_loss_avg 13.608418464660645 psnr 23.369504928588867 psnr_avg 22.018949508666992
>12420, d[0.738] g[10.349] d_loss_avg 0.2569042146205902 g_loss_avg 12.793636322021484 psnr 23.611696243286133 psnr_avg 22.41713523864746
>12421, d[0.431] g[10.094] d_loss_avg 0.19764366745948792 g_loss_avg 12.253744125366211 psnr 23.82022094726

>Saved: /content/drive/MyDrive/Image_Denoising/plot_GAN_L4_WGP_Transpose012804.png and /content/drive/MyDrive/Image_Denoising/GAN_model_GITHUB_L4_WGP_Transpose012804
After 12804th step psnr_avg 23.297142028808594 and psnr_val 20.977195739746094
D_LOSS_VAL 0.017478760331869125 G_LOSS_VAL 16.95907211303711 PSNR_VALIDATION is: 20.812864303588867
>12805, d[0.005] g[10.794] d_loss_avg 0.005487763788551092 g_loss_avg 10.794109344482422 psnr 24.041492462158203 psnr_avg 24.041492462158203
>12806, d[0.065] g[15.285] d_loss_avg 0.145942822098732 g_loss_avg 13.039441108703613 psnr 21.35079574584961 psnr_avg 22.696144104003906
>12807, d[0.016] g[8.806] d_loss_avg 0.24425210058689117 g_loss_avg 11.628382682800293 psnr 25.112812042236328 psnr_avg 23.501699447631836
>12808, d[0.110] g[9.605] d_loss_avg 0.23820455372333527 g_loss_avg 11.122451782226562 psnr 24.299537658691406 psnr_avg 23.70115852355957
>12809, d[0.400] g[12.197] d_loss_avg 0.20298251509666443 g_loss_avg 11.337413787841797 psnr 22.8658

>Saved: /content/drive/MyDrive/Image_Denoising/plot_GAN_L4_WGP_Transpose013192.png and /content/drive/MyDrive/Image_Denoising/GAN_model_GITHUB_L4_WGP_Transpose013192
After 13192th step psnr_avg 23.081668853759766 and psnr_val 20.667407989501953
D_LOSS_VAL 0.044952891767024994 G_LOSS_VAL 17.184104919433594 PSNR_VALIDATION is: 20.756175994873047
>13193, d[0.044] g[10.398] d_loss_avg 0.04408538341522217 g_loss_avg 10.398301124572754 psnr 23.70717430114746 psnr_avg 23.70717430114746
>13194, d[0.007] g[12.877] d_loss_avg 0.0637575089931488 g_loss_avg 11.637617111206055 psnr 22.223834991455078 psnr_avg 22.965503692626953
>13195, d[0.010] g[10.192] d_loss_avg 0.05784277617931366 g_loss_avg 11.155838012695312 psnr 23.086925506591797 psnr_avg 23.0059757232666
>13196, d[0.753] g[9.857] d_loss_avg 0.2832332253456116 g_loss_avg 10.831238746643066 psnr 24.088441848754883 psnr_avg 23.276592254638672
>13197, d[0.106] g[11.194] d_loss_avg 0.13175718486309052 g_loss_avg 10.903874397277832 psnr 22.95503

KeyboardInterrupt: 